
# MLflow Promotion Test Notebook

This notebook helps you **inspect MLflow runs** and **exercise the promotion flow** that your `scripts/promote_best.py` implements.

> **What you'll be able to do here**
> 1. Point to your running MLflow Tracking Server.
> 2. Inspect runs in the `credit_risk_training` experiment (or another one you specify).
> 3. Verify each run actually logged a model artifact and discover the true `artifact_path` from the `mlflow.log-model.history` tag.
> 4. (Optionally) Register and Promote the best run **directly from the notebook** (mimicking your script).


In [5]:
%pip install -q mlflow scikit-learn xgboost pandas numpy
# --- Configure here ---
MLFLOW_TRACKING_URI = "http://a2-mlflow:5000"   # or "http://localhost:5000" from host
EXPERIMENT_NAME     = "credit_risk_training"
TRAIN_DATE          = "2024-03-01"              # must match tags.train_date in your runs
METRIC_KEY          = "auc_oot"                 # metric to maximize when picking 'best'
MODEL_NAME          = "credit_risk_model"
PROMOTE_STAGE       = "Production"              # "Staging", "Production", "Archived", "None"
ARCHIVE_EXISTING    = True                      # archive previous versions in target stage
DRY_RUN             = True                      # True = preview only; False = actually register/promote


Note: you may need to restart the kernel to use updated packages.


In [6]:

import os, json
import pandas as pd
import mlflow
from mlflow.tracking import MlflowClient

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
client = MlflowClient()

exp = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
assert exp is not None, f"Experiment '{EXPERIMENT_NAME}' not found at {MLFLOW_TRACKING_URI}"
print("Tracking URI:", MLFLOW_TRACKING_URI)
print("Experiment:", exp)


Tracking URI: http://a2-mlflow:5000
Experiment: <Experiment: artifact_location='/tmp/mlflow/artifacts/1', creation_time=1761997655856, experiment_id='1', last_update_time=1761997655856, lifecycle_stage='active', name='credit_risk_training', tags={}>


In [7]:

df = mlflow.search_runs(
    experiment_ids=[exp.experiment_id],
    filter_string="attributes.status = 'FINISHED'",
    order_by=["start_time DESC"],
    max_results=200,
)
cols = [
    "run_id","tags.train_date","tags.flavor","status",
    "metrics.auc_train","metrics.auc_test","metrics.auc_oot"
]
print("Total FINISHED runs:", len(df))
try:
    display(df[cols].head(20))
except Exception:
    print(df.head(5))


Total FINISHED runs: 6


,run_id,tags.train_date,tags.flavor,status,metrics.auc_train,metrics.auc_test,metrics.auc_oot
0,b27be6b792c54770af5f445705baf1fa,2024-03-01,xgboost,FINISHED,0.939077,0.822929,0.879334
1,7f686a31f82a48028dd926033003413a,2024-03-01,logreg,FINISHED,0.830940,0.808425,0.841697
2,3f6d72694dd74a6c919f3a6f6bc24553,2024-03-01,xgboost,FINISHED,0.939077,0.822929,0.879334
3,2a39616928214c9793a23c74aaddc7a0,2024-03-01,logreg,FINISHED,0.830940,0.808425,0.841697
4,84c9a3b883224deaa3c3f124e9da2bdd,2024-03-01,xgboost,FINISHED,0.939077,0.822929,0.879334
5,96dfdbee0f8d4d1e9e4c2217edadd00b,2024-03-01,logreg,FINISHED,0.830940,0.808425,0.841697


In [8]:

flt = (df["tags.train_date"] == TRAIN_DATE)
subset = df.loc[flt].copy()
if subset.empty:
    raise SystemExit(f"No FINISHED runs found with tags.train_date == {TRAIN_DATE}")

subset[f"metrics.{METRIC_KEY}"] = pd.to_numeric(subset.get(f"metrics.{METRIC_KEY}"), errors="coerce")
subset = subset.dropna(subset=[f"metrics.{METRIC_KEY}"]).sort_values(by=f"metrics.{METRIC_KEY}", ascending=False)
if subset.empty:
    raise SystemExit(f"No runs have metric '{METRIC_KEY}' for train_date {TRAIN_DATE}")

best = subset.iloc[0]
run_id = best.run_id
best_metric = float(best[f"metrics.{METRIC_KEY}"])
print(f"Best run: {run_id} | {METRIC_KEY}={best_metric:.6f}")
try:
    display(best.to_frame())
except Exception:
    print(best)


Best run: b27be6b792c54770af5f445705baf1fa | auc_oot=0.879334


,0
run_id,b27be6b792c54770af5f445705baf1fa
experiment_id,1
status,FINISHED
artifact_uri,/tmp/mlflow/artifacts/1/b27be6b792c54770af5f44...
start_time,2025-11-01 12:46:58.962000+00:00
...,...
tags.source,airflow
tags.flavor,xgboost
tags.mlflow.source.name,train_xgboost_ml.py
tags.mlflow.user,airflow


In [9]:

run = client.get_run(run_id)
hist_tag = run.data.tags.get("mlflow.log-model.history")
print("Has mlflow.log-model.history tag?", bool(hist_tag))

artifact_path = None
if hist_tag:
    try:
        hist = json.loads(hist_tag)
        print("Logged models (most recent last):")
        for e in hist:
            print(" -", e.get("artifact_path"))
        for e in reversed(hist):
            cand = e.get("artifact_path")
            if cand:
                try:
                    _ = client.list_artifacts(run_id, cand)
                    artifact_path = cand
                    break
                except Exception:
                    pass
    except Exception as e:
        print("Failed to parse log-model history:", e)

if not artifact_path:
    artifact_path = "model"
print("Chosen artifact_path:", artifact_path)

arts = client.list_artifacts(run_id, artifact_path)
print("Artifact entries under this path:")
for a in arts:
    print(" *", a.path)


Has mlflow.log-model.history tag? False
Chosen artifact_path: model
Artifact entries under this path:


In [10]:

from mlflow.exceptions import MlflowException

src_uri = f"runs:/{run_id}/{artifact_path}"
print("Registering source:", src_uri)

existing = [
    mv for mv in client.search_model_versions(f"name='{MODEL_NAME}'")
    if mv.run_id == run_id and mv.source == src_uri
]
if existing:
    mv = sorted(existing, key=lambda m: int(m.version))[-1]
    print(f"Reuse existing version: {MODEL_NAME} v{mv.version}")
else:
    if DRY_RUN:
        print(f"[DRY RUN] Would register model from {src_uri} as '{MODEL_NAME}'")
        mv = None
    else:
        mv = mlflow.register_model(src_uri, MODEL_NAME)
        print(f"Registered: {MODEL_NAME} v{mv.version}")

if mv is not None and not DRY_RUN:
    client.transition_model_version_stage(
        name=MODEL_NAME,
        version=mv.version,
        stage=PROMOTE_STAGE,
        archive_existing_versions=bool(ARCHIVE_EXISTING),
    )
    print(f"Transitioned {MODEL_NAME} v{mv.version} -> {PROMOTE_STAGE} (archive_existing={ARCHIVE_EXISTING})")
else:
    print("[DRY RUN] Skipping stage transition.")


Registering source: runs:/b27be6b792c54770af5f445705baf1fa/model
[DRY RUN] Would register model from runs:/b27be6b792c54770af5f445705baf1fa/model as 'credit_risk_model'
[DRY RUN] Skipping stage transition.


In [11]:
from mlflow.tracking import MlflowClient
c = MlflowClient()
r = c.get_run("84c9a3b883224deaa3c3f124e9da2bdd")
print("artifact_uri:", r.info.artifact_uri)

artifact_uri: /tmp/mlflow/artifacts/1/84c9a3b883224deaa3c3f124e9da2bdd/artifacts


In [ ]:
import mlflow.pyfunc
mvs = sorted(client.search_model_versions(f"name='{MODEL_NAME}'"), key=lambda m: int(m.version))
print("Model versions found:", [m.version for m in mvs])
prod = [m for m in mvs if m.current_stage == PROMOTE_STAGE]
uri = None
if prod:
    uri = f"models:/{MODEL_NAME}/{PROMOTE_STAGE}"
elif mvs:
    uri = f"models:/{MODEL_NAME}/{mvs[-1].version}"

print("Load URI:", uri)
if uri:
    m = mlflow.pyfunc.load_model(uri)
    print("Loaded model type:", type(m))
else:
    print("No model version available to load yet.")
